In [1]:
import sys
import re 

In [2]:
def arrayPositions(data):
    bracketMatches = list(re.finditer('\\[|\\]', data))
    prevBracket = ''
    openBracketPositions = []
    openBracketDuplicates = []
    closeBracketPositions = []
    closeBracketDuplicates = []
    for match in bracketMatches:
        bracketIndex = match.span()[0]
        bracket = data[bracketIndex]
        if(bracket == '['):
            openBracketPositions.append(bracketIndex)
            if(bracket == prevBracket):
                openBracketDuplicates.append(len(openBracketPositions) - 1)
        else:
            closeBracketPositions.append(bracketIndex)
            if(bracket == prevBracket):
                closeBracketDuplicates.append(len(closeBracketPositions) - 1)
        prevBracket = bracket
    for i in reversed(openBracketDuplicates):
        del openBracketPositions[i]
    for i in reversed(closeBracketDuplicates):
        del closeBracketPositions[i]
    return openBracketPositions, closeBracketPositions   

In [4]:
fName = 'He_SAS_191213_0935.json'
with open(fName, 'r', encoding="utf-8") as JSONfile:
    data = JSONfile.read()
    print(data[:2000])

{
    "FileVersion":0.00000000000000,
    "Summary":{
        "Apparatus":0,
        "ExperimentType":"Symmetric Angular Scan",
        "Species":"He",
        "Description":"Angular sweep to check health of analysers' spot welds.",
        "CountType":"Coincidence Counts",
        "StartDate":"191213_0935",
        "EstimatedExperimentDuration":"70:00",
        "ActualExperimentDuration":"",
        "StepTime":3000,
        "NumberOfSteps":84,
        "TAC":{
            "FullScale (ns)":1000.00000000000,
            "NumberOfBins":1000,
            "WindowStart (ns)":410,
            "WindowStop (ns)":610
        },
        "FCup":{
            "Current (µA)":0.646306674862373,
            "Voltage (V)":300.000000000000
        },
        "Pressure(Torr)":5.76300081042067E-6,
        "Gun":{
            "Energies":[
                45.2400000000000
            ],
            "Angles":[
                0.00000000000000
            ]
        },
        "A1":{
            "RSE":[
      

In [10]:
s = '[0, 1, 2]'
s = "[{'1':'one', '2' : 'two'},{'3':'sdfone', '4' : 'sdftwo'}]"
tList = eval(s)
for t in tList:
    print(type(t))

<class 'dict'>
<class 'dict'>


In [37]:
# build nested list of array positions
def arrayNestedPositions(data):
    bracketMatches = list(re.finditer('\\[|\\]', data))
#     prevBracket = ''
    bracketPositions = [[]]
    for match in bracketMatches:
        bracketIndex = match.span()[0]
        bracket = data[bracketIndex]
        lastListLen = len(bracketPositions[-1])
        if(bracket == '['):
            if lastListLen % 2:
                # length is odd so the previous bracket must ALSO have been an open bracket,
                # therefore, we are now in a nested array.
                bracketPositions[-1].append(bracketIndex)
            elif lastListLen == 0:
                bracketPositions[-1] = [bracketIndex]
            else:
                bracketPositions.append([bracketIndex])
        else:
            if lastListLen % 2:
                # the previous close bracket closed an array, therefore
                # we are leaving an array
                bracketPositions.append([bracketIndex])
            else:
                bracketPositions[-1].append(bracketIndex)
    return bracketPositions

arrayNestedPositions(data)

[[828],
 [875, 899, 946, 992, 1039, 1098, 3967, 4013, 4060, 4119, 6988, 7022],
 [7036],
 [9905],
 [9915, 9935],
 [13865],
 [13879,
  13903,
  34927,
  38868,
  38882,
  38906,
  59922,
  63862,
  63876,
  63900,
  84905,
  88846,
  88860,
  88884,
  109887,
  113830,
  113844,
  113868,
  134850,
  138792,
  138806,
  138830,
  159325,
  163267,
  163281,
  163305,
  183411,
  187354,
  187368,
  187392,
  207441,
  211383,
  211397,
  211421,
  231463,
  235404,
  235418,
  235442,
  255485,
  259427,
  259441,
  259465,
  279534,
  283477,
  283491,
  283515,
  303838,
  307780,
  307794,
  307818,
  327978,
  331922,
  331936,
  331960,
  352010,
  355954,
  355968,
  355992,
  376054,
  379998,
  380012,
  380036,
  400150,
  404092,
  404106,
  404130,
  424819,
  428762,
  428776,
  428800,
  449816,
  453760,
  453774,
  453798,
  474817,
  478759,
  478773,
  478797,
  499816,
  503758,
  503772,
  503796,
  524815,
  528757,
  528771,
  528795,
  549814,
  553758,
  553772,
  

In [ ]:
def fromNestedDict(data, fields):
    ''' 
    Recursive return from dictionary. Feed it a dictionary (JSON) and a list of key strings and it will return the value.
    '''
    firstDict = fields[0]
    remainingDicts = fields[1:]

    if remainingDicts:
        return fromNestedDict(data[firstDict], remainingDicts)
    else:
        return data[firstDict]

In [27]:
a = 13
if a%2:
    print('odd')
else:
    print('even')

odd


In [31]:
b = [[]]
len(b[-1])

0

In [15]:
# Find the numeric arrays and return their open and close positions inner arrays
def arrayPositionsAndDepths(data):
    bracketMatches = list(re.finditer('\\[|\\]', data))
    prevBracket = ''
    openBracketPositions = []
    depthList = []
    depth = 0
    openBracketDuplicates = []
    closeBracketPositions = []
    closeBracketDuplicates = []
    for match in bracketMatches:
        bracketIndex = match.span()[0]
        bracket = data[bracketIndex]
        if(bracket == '['):
            openBracketPositions.append(bracketIndex)
            if(bracket == prevBracket):
                depth += 1
                openBracketDuplicates.append(len(openBracketPositions) - 1)
        else:
            closeBracketPositions.append(bracketIndex)
            if(bracket == prevBracket):
                depth -= 1
                closeBracketDuplicates.append(len(closeBracketPositions) - 1)
        depthList.append(depth)
        prevBracket = bracket
    return depthList

l = arrayPositionsAndDepths(data)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 